In [ ]:
import os
import numpy as np
import keras
import librosa
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from keras.models import Model, Sequential
from keras.layers import Input, Conv2D, Flatten, GlobalMaxPooling2D , GlobalMaxPooling1D, MaxPooling2D, MaxPooling1D, Activation, BatchNormalization, GlobalAveragePooling2D, GlobalMaxPool2D, concatenate, Dense, Dropout
from keras.layers import  Conv2D, MaxPooling2D, UpSampling2D, Lambda, Reshape
from keras.layers import Input, GRU, RepeatVector, BatchNormalization, TimeDistributed, Conv1D
from keras.layers import GlobalAveragePooling1D, LSTM
from keras.optimizers import Adam
from tensorflow.python.keras.utils import to_categorical

from sklearn.metrics import classification_report,accuracy_score
from keras.models import model_from_json

from scipy.io import wavfile
from tqdm import tqdm
from pydub import AudioSegment


path = "data/train/audio"

In [ ]:
models = []
weights = []
for file in glob.glob("models/*.json"):
    models.append(file)
for file in glob.glob("models/*.h5"):
    weights.append(file)

In [ ]:
## Choose a model that you want to run and the appropriate weight only
model_to_run = models[4]
weights_to_use = weights[1]

In [ ]:
json_file = open(model_to_run, 'r')
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)

model.load_weights(weights_to_use)

In [ ]:
# Get the training data 
# Be sure to check out augmentation = False or True, based on the model you have chosen
X_train, X_test, y_train, y_test = def prepare_training_data(data, augmentation = False):

In [ ]:
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

In [ ]:
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

In [ ]:
# Do not reshape if you have chosen 1D models
X_train_2d = X_train.reshape(X_train.shape[0], 12, 51, 1)
X_test_2d = X_test.reshape(X_test.shape[0], 12, 51, 1)

In [ ]:
def accuracy(X_test,y_test_cat):
    y_pred = model.predict(X_test)
    y_pred_val = []
    for row in y_pred:
        y_pred_val.append(np.argmax(row))
    y_pred_val = to_categorical(y_pred_val)
    return accuracy_score(y_test_cat,y_pred_val),y_pred_val

In [ ]:
def get_report(X_train, X_test, y_train_cat, y_test_cat,X_train_2d,X_test_2d,TwoD_layer = False):
    if TwoD_layer == True:
        print('Test accuracy is',accuracy(X_test_2d,y_test_cat)[0] )
        print('Train accuracy is',accuracy(X_train_2d,y_train_cat)[0])
        y_pred_val = accuracy(X_test_2d,y_test_cat)[1]
    else:
        print('Test accuracy is',accuracy(X_test,y_test_cat) )
        print('Train accuracy is',accuracy(X_train,y_train_cat))
        y_pred_val = accuracy(X_test_2d,y_test_cat)[1]
    print("classification report on validation data \n")
    labels_actual = 'yes no up down left right on off stop go silence unknown'.split()
    print(classification_report(y_test_cat,y_pred_val, target_names= labels_actual))

In [ ]:
get_report(X_train, X_test, y_train_cat, y_test_cat,X_train_2d,X_test_2d,True)

In [ ]:
mfcc_test = np.load('mfcc_test.npy')

In [ ]:
# Do not run this step if your chosen model has 1D convolutions
mfcc_test = mfcc_test.reshape(-1,12,51,1)

In [ ]:
# predictions on test set
y_pred = model.predict(mfcc_test)
y_pred_val = []
for row in (y_pred):
    y_pred_val.append(np.argmax(row))

In [ ]:
labels_actual_dict = {0: 'right', 1:'yes', 
                     2:'no',3: 'up', 4: 'down', 5:'left', 6: 'on', 7: 'off',
                     8: 'stop', 9: 'go', 10: 'silence', 11: 'unknown'}

In [ ]:
test_files = np.load('test_files.npy')

In [ ]:
with open('submission.csv','a') as f:
    columnTitleRow = "fname,label\n"
    f.write(columnTitleRow)
    
    for i in range(len(y_pred_val)):
        answer = labels_actual_dict[y_pred_val[i]]
        row = '{},{}\n'.format(test_files[i],answer)
        f.write(row)